In [19]:
!pip install accelerate peft bitsandbytes transformers trl

In [20]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [22]:
def finetune_llama_v2():
    data = load_dataset("timdettmers/openassistant-guanaco", split="train")
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-hf", quantization_config=bnb_config, device_map = {"":0}
    )
    model.config.use_cache = False # to ensure training mode
    model.config.pretraining_tp = 1
    peft_config = LoraConfig(
        r=64, lora_alpha =32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
    training_arguments = TrainingArguments(
        output_dir="llama2_finetuned_chatbot",
        per_device_train_batch_size=6,
        gradient_accumulation_steps=4,
        optim = "paged_adamw_8bit",
        learning_rate = 2e-4,
        lr_scheduler_type = "linear",
        save_strategy = "epoch",
        logging_steps= 10,
        num_train_epochs=1,
        max_steps=10,
        fp16=True,
        push_to_hub=True
    )
    trainer = SFTTrainer(
        model = model,
        train_dataset=data,
        peft_config = peft_config,
        dataset_text_field = "text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length = 512
    )
    trainer.train()
    trainer.push_to_hub()

In [24]:
if __name__ == "__main__":
    finetune_llama_v2()

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at meta-llama/Llama-2-7b-hf were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'mode

AssertionError: libcuda.so cannot found!
Possible files are located at ['/usr/lib/x86_64-linux-gnu/libcuda.so.1'].Please create a symlink of libcuda.so to any of the file.